In [ ]:
import nltk
nltk.download('punkt')

import cv2
import numpy as np
from numpy.core.defchararray import join, mod
import pandas as pd
import matplotlib.pyplot as plt
import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2
import os
import torch
from torch._C import device
import torch.autograd.function as F
from torch.utils.data import Dataset, DataLoader
import torchvision
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.models.detection import FasterRCNN
from torchvision.models.detection import fasterrcnn_resnet50_fpn
from torchvision.models.detection.rpn import AnchorGenerator
from PIL import Image
from torchvision import transforms as torchtrans  
import re
import nltk
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
import xgboost as xgb
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import pickle

nltk.download('stopwords')
ar_stopwords = set(nltk.corpus.stopwords.words('arabic'))
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

def preprocess_text(text):
  # Preprocess the text data
  # Normalize the text by removing diacritics and converting characters to their basic form
  text = re.sub(r'[\u064b-\u065f\u0640]','', text)
  text = re.sub(r'[إأٱآا]', 'ا', text)
  text = re.sub(r'[ئؤ]', 'ء', text)
  # Remove non-Arabic characters and numbers
  text = re.sub(r'[^\u0600-\u06FF\s]','', text)
  text = re.sub(r'\d+','', text)
  # Remove stop-words
  tokens = [token for token in nltk.word_tokenize(text) if token not in ar_stopwords]
  text = ' '.join(tokens)
  return text

def process_image(img):
  img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB).astype(np.float32)
  img_res = img_rgb/255
  img_res = torch.as_tensor(img_res).to(device)
  img_res = img_res.permute(2, 0, 1)
  return img_res=

with open('model.pkl', 'rb') as f:
    text_model = pickle.load(f)
text_vectorizer = pickle.load(open('vectorizer.pkl', 'rb'))

weapons_model_filename = "weapon_trained_model-1.0a.pt"

cpu_device = torch.device("cpu")
weapons_model= torch.load(weapons_model_filename, map_location=torch.device('cpu'))
weapons_model.eval()

def predict_image(img):
  input = []
  input.append(img)
  outputs = weapons_model(input)
  outputs_list = outputs[0]['scores'].tolist()
  if len(outputs_list) > 0: 
    return max(outputs_list) 
  else: 
    return 0

def predict_text(text):
  new_sentence = preprocess_text(text)
  new_sentence_vectorized = text_vectorizer.transform([new_sentence])
  predicted_label = text_model.predict(new_sentence_vectorized)[0]
  return int(predicted_label)

import tweepy
import requests
from textblob import TextBlob
import cv2

# Twitter API credentials
consumer_key = "***"
consumer_secret = "***"
access_token = "***"
access_token_secret = "***"

# Connect to Twitter API
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth)

username = input("Enter the username without '@': ")

user = api.get_user(screen_name=username)
tweets = api.user_timeline(screen_name=username, count=5)
favorites = api.get_favorites(screen_name=username, count=1)
user_profile_image_url = user.profile_image_url_https
response = requests.get(user_profile_image_url)
with open(f"{user.screen_name}_profile_image.jpg", "wb") as f:
    f.write(response.content)
profile_image = cv2.imread(f"{user.screen_name}_profile_image.jpg")
profile_image_processed = process_image(profile_image)
profile_image_score = 1 - predict_image(profile_image_processed)

text_score = 0
for tweet in tweets:    
  text_score += predict_text(tweet.text)
text_score = len(tweets) - text_score



# print("Profile image score:", profile_image_score)
# print("Profile image score:", int(profile_image_score))
# print("Tweets score:", text_score)



if int(profile_image_score) <= 0:
    print("• Warning: A negative image has been detected!")
    

if text_score <= 4:
    print("• Warning: A negative tweet has been detected!")
    
if int(profile_image_score) <= 0 and text_score <= 4 :
    print("• Warning: There is a possibility that the person may have criminal motives!")
    
if int(profile_image_score) == 1:
    print("• No negative image has been detected.")
    
if text_score == 5:
    print("• No negative tweet has been detected.")

 

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


[11:09:13] WARNING: ../src/learner.cc:553: 
  If you are loading a serialized model (like pickle in Python, RDS in R) generated by
  older XGBoost, please export the model by calling `Booster.save_model` from that version
  first, then load it back in current version. See:

    https://xgboost.readthedocs.io/en/latest/tutorials/saving_model.html

  for more details about differences between saving model and serializing.

Enter the username without '@': Anas_5x
Profile image score: 1
Profile image score: 1
Tweets score: 5
• No negative image has been detected.
• No negative tweet has been detected.
